In [33]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
#!/usr/bin/env python
# coding: utf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

import src.features.pabellon as pb

# Configuración de gráficos
sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

ANIO_INICIO = 2017
ANIO_TERMINO = 2035
COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

In [35]:
# Cargar datos
casos_area_de_influencia = pb.cargar_casos_area_de_influencia(
    "../data/interim/casos_teoricos_diagnosticos.xlsx"
)
porcentajes_de_quirurgicos = pb.cargar_porcentajes_de_quirurgicos(
    "../data/external/incidencias_y_prevalencias_INT.xlsx"
)

# Calcular casos quirúrgicos
DIAGS_AREA_DE_INFLUENCIA = casos_area_de_influencia.index
casos_quirurgicos = pb.calcular_casos_quirurgicos(
    casos_area_de_influencia, porcentajes_de_quirurgicos
)

Casos del área de influencia cargados y formateados:
+---+-------------+-------------------------+-------------+----------------------+-----------------+-----------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|   | Diagnostico | Diagnosticos Contenidos | Estadística | Casos (Cada 100.000) | Edad Incidencia |                                  

In [40]:
# Lee la base de GRD unida con Pabellon
grd_y_pabellon = pd.read_csv("../data/raw/5_duracion_int_q/grd_interno_y_pabellon.csv")
grd_y_pabellon["duracion"] = pd.to_timedelta(grd_y_pabellon["duracion"])

# Extrae los diagnosticos que se van a reasignar
diagnosticos_a_reasignar = casos_area_de_influencia.dropna(subset="Diagnosticos Contenidos")
diagnosticos_a_reasignar = diagnosticos_a_reasignar["Diagnosticos Contenidos"].str.split(", ")
diagnosticos_a_reasignar = diagnosticos_a_reasignar.reset_index()

# Reasigna los diagnosticos en la base de datos original
for row in diagnosticos_a_reasignar.itertuples():
    diagnostico_nuevo = row[1]
    diagnosticos_antiguos = row[2]
    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    grd_y_pabellon["diag_01_principal_cod"] = grd_y_pabellon["diag_01_principal_cod"].replace(
        diagnosticos_antiguos, diagnostico_nuevo
    )

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q676', 'Q677', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX
Cambiando ['I052', 'I080', 'I081'] a I051
Cambiando ['I352'] a I350


In [41]:
# Obtiene el resumen de duraciones de las int. q por diagnostico acumuladas
resumen_duraciones_int_q_acumuladas = (
    grd_y_pabellon.query("ano_de_egreso >= 2019 ")
    .groupby(["diag_01_principal_cod"])["duracion"]
    .describe()
)

# Obtiene el resumen de duraciones para los diagnosticos relevantes
resumen_duraciones_diags_relevantes = resumen_duraciones_int_q_acumuladas[
    resumen_duraciones_int_q_acumuladas.index.isin(DIAGS_AREA_DE_INFLUENCIA)
]["75%"]

In [43]:
# Calcular tiempo utilizado en pabellón en horas
tiempo_utilizado_pabellon_horas = pb.calcular_tiempo_utilizado_pabellon(
    casos_quirurgicos, resumen_duraciones_diags_relevantes
)

# Calcular horas laborales
horas_laborales = pb.calcular_horas_laborales(ANIO_INICIO, ANIO_TERMINO, 12)

# Calcular cantidad de pabellones necesarios
cantidad_de_pabellones_necesarios = pb.calcular_cantidad_de_pabellones_necesarios(
    tiempo_utilizado_pabellon_horas, horas_laborales
)

Tiempo utilizado en pabellón calculado (en horas):
+------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|      |        2017        |        2018        |       2019        |        2020        |        2021        |        2022        |    2023    |       2024        |        2025        |       2026        |       2027        |       2028        |       2029        |        2030        |        2031        |        2032        |       2033        |        2034        |        2035        |
+------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------------+-----------------